In [1]:
import torch
import numpy as np
import os
import shutil
from datetime import datetime
import math
from math import exp
from random import seed
import random as rand
import matplotlib.pyplot as plt
import pickle 

import sys
sys.path.append('../common/')
from utils import *
from loo_cv import *

In [2]:
seed=0
torch.manual_seed(seed)

In [3]:
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
from torch.jit import script, trace       # hybrid frontend decorator and tracing jit
from torch.optim.lr_scheduler import StepLR

In [4]:
class Net(nn.Module):
    def __init__(self,N=10):
            super(Net, self).__init__()
            
            nInputs = int(N*N/2-N/2)
            
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(in_features=nInputs, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=32),
                nn.ReLU(),
                nn.Linear(in_features=32, out_features=16),
                nn.ReLU(),
                nn.Linear(in_features=16, out_features=1)
            )
            # map to positive
            
            
    def forward(self, x):
            logits = self.linear_relu_stack(x)
            return logits

# generate features

In [5]:
def upper_half(A):
    n = A.shape[0]
    return 1/A[np.triu_indices(n, k = 1)]

In [6]:
# network to be tested:
directory = "plots_heat1d/"
if not os.path.exists(directory):
    os.makedirs(directory)

n=10
path =   '../network/results_sorted_1d_2d/best_model.pt'
model=Net()
model.load_state_dict(torch.load(path))
model.eval()

Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=45, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=16, bias=True)
    (9): ReLU()
    (10): Linear(in_features=16, out_features=1, bias=True)
  )
)

# heat one dimension

In [7]:
# one dimensional time-dependent problem, for example
# u_{t}=\beta*u_{xx}, x \in [a,b], t \in [0,1]
# u(x,0)= initial_condition, x \in [a,b]
# u(a,t)=u_exact(a,t), u(b,t)=u_exact(b,t), t \in [0,1]

In [9]:
import math
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from datetime import datetime
import math
from math import exp
from random import seed
import random as rand
from scipy.spatial import distance

def phi(f, x, y):
    ss = (1 + (f * np.linalg.norm(x-y)) ** 2.0) ** (-0.5)
    return ss
def phixx(f, x, y):
    xx = -(f**2.0)*((1 + (f * np.linalg.norm(x-y)) ** 2.0) ** (-1.5))+3.0*(f**4.0)*((np.linalg.norm(x-y)) ** 2.0)*((1 + (f * np.linalg.norm(x-y)) ** 2.0) ** (-2.5))
    return xx
    
def solve_1d_heat(initial_condition,u_exact,T,M,N,n,x,a,b):
    dt=T/M
    u=np.zeros((4,M+1,N)) # approximated values for 4 strategies: NN, Hardy, Franke and modified Franke
    for count in range(4):
        for i in range(N):    
            u[count,0,i]=initial_condition(x[i])
    true_vals=np.zeros((M+1,N))
    for j in range(M+1): 
        for i in range(N):
            true_vals[j,i]=u_exact(x[i],j*dt)        

    distance=np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            distance[i][j]=(abs(x[i]-x[j]))

    closest = np.argsort(distance, axis=1)

    boundary_indices=[]
    non_boundary_indices=[]
    for i in range(N):
        if (not np.isclose(x[i],a)) and (not np.isclose(x[i],b)):
            non_boundary_indices.append(i)
        else:
            boundary_indices.append(i)
            
            
    for k in range(M):
        D=np.zeros((4,N,N))
        
        for y in range(N):
            x_local=list()
            
            for q in range(n):
                x_local.append(x[closest[y][q]])           
            
            # Get the indices that would sort the array based on the distances
            sorted_indices = np.argsort(x_local)
            
            x_local.sort()
            x_local1=np.array([x_local[idx] for idx in range(n)])    
            x_local_mod=np.zeros(2*x_local1.shape[0])
            x_local_mod[::2] =x_local1      
            x_local_mod=x_local_mod.reshape(n,2)

            # Compute shape parameter obtained with Hardy, Franke and modified Franke strategies
            d=0
            for i in range(n):
                if i==0:
                    d=d+x_local[i+1]-x_local[i]
                elif i==n-1:
                    d=d+x_local[i]-x_local[i-1]
                else:
                    d=d+min(x_local[i]-x_local[i-1],x_local[i+1]-x_local[i])
            d=d/n
            shape_hardy=1/(0.815*d) #Hardy
            shape_franke=0.8*(10**(1/2))/(x_local[9]-x_local[0]) #Franke
            shape_mfranke=0.8*(10**(1/4))/(x_local[9]-x_local[0]) #Modified Franke
            
            X=generate_distance_from_coordinates(x_local_mod)    
            training_set_distances_flatten = upper_half(X).reshape((1,int(n*n/2-n/2)))
            scaled_training_set_flat = training_set_distances_flatten   
            scaled_training_set_tensor = torch.tensor(scaled_training_set_flat,dtype=torch.float)  
            shape_optimisation= model(scaled_training_set_tensor).item()

            for count,shape in enumerate([shape_optimisation,shape_hardy,shape_franke,shape_mfranke]):
        
                L=np.zeros((n+1,n+1))
                Lxx=np.zeros((n+1,1)) 
                for j in range(n):
                    for i in range(n):
                
                        L[i, j] = phi(shape, x_local[i], x_local[j])
                    
                        L[i,n]=1.0
                    L[n,j]=1.0
        
                    Lxx[j,0] = phixx(shape, x[y], x_local[j])
       
                L[n,n]=0.0
                Lxx[n,0]=0.0
            
                w=np.linalg.solve(L,Lxx)

                D[count][y,closest[y][:n][sorted_indices]]=w[:n].reshape(n,)           
        for count in range(4):
            boundary_condition=np.matmul(D[count][np.ix_(non_boundary_indices,boundary_indices)],true_vals[k,boundary_indices])
        
            if k==0:        
                u[count][k+1,non_boundary_indices]=np.linalg.solve(np.identity(len(non_boundary_indices))-dt*beta*D[count][np.ix_(non_boundary_indices,non_boundary_indices)],u[count][k,non_boundary_indices]+dt*beta*boundary_condition)
            else:
                u[count][k+1,non_boundary_indices]=np.linalg.solve(np.identity(len(non_boundary_indices))-dt*beta*D[count][np.ix_(non_boundary_indices,non_boundary_indices)]*2.0/3.0,4.0*u[count][k,non_boundary_indices]/3.0-u[count][k-1,non_boundary_indices]/3.0+dt*beta*boundary_condition*2.0/3.0)   
            u[count][k+1,boundary_indices]=true_vals[k+1,boundary_indices]
            

    error=[0 for i in range(4)]
    for count in range(4):
        for j in range(M+1): 
            for i in range(N):
                error[count]=error[count]+(true_vals[j,i]-u[count][j,i])**2.0
        
    return [math.sqrt(err/((M+1)*N))  for err in error]
    

In [ ]:
def u_exact1(x,t):
    Answer=0
    for k in range(1,50):
        Answer +=(-4*((-1)**k-1)*math.sin(k*math.pi*x)*math.exp(-(k*(math.pi))**2*t)/((k*(math.pi))**3))
    return(Answer)

def u_exact2(x,t):    
    Answer =6.0*math.sin((math.pi)*x)*math.exp(-beta*((math.pi)**2.0)*t)
    return(Answer)


initial_condition1=lambda x: -x**2+x
initial_condition2=lambda x: 6.0*math.sin((math.pi)*x)

beta=1.0 #heat equation: u_{t}=\beta*u_{xx}
T=1.0
M=1000
n=10
a=0
b=1
no_testcase=1

for initial_condition,u_exact in zip([initial_condition1,initial_condition2],[u_exact1,u_exact2]):
    no_test=no_testcase
    
    for uniform in ['true', 'false']:
        no_points=[10,19,37,73,145]
        optimisation_error=[]
        hardy_error=[]
        franke_error=[]
        mfranke_error=[]
        for N in no_points:
            x = np.zeros((1,N))
            if uniform=='true':
                x=[i/(N-1) for i in range(N)]
            elif uniform =='false':
                x=[0.5+0.5*np.cos((2*k-1)*np.pi/(2.0*N)) for k in range(1,N-1)]
                x.append(a)
                x.append(b)
                x.sort()
                

            error=solve_1d_heat(initial_condition,u_exact,T,M,N,n,x,a,b)
            
            optimisation_error.append(error[0])
            hardy_error.append(error[1])
            franke_error.append(error[2])
            mfranke_error.append(error[3])
        fig=plt.figure()
        plt.loglog(no_points,hardy_error,color='orange',marker='o')
        plt.loglog(no_points,franke_error,color='g',marker='o')
        plt.loglog(no_points,mfranke_error,color='purple',marker='o')
        plt.loglog(no_points,optimisation_error,color='r',marker='o')

        plt.tick_params(axis='both', which='major', labelsize=13, width=2, length=5)

        plt.xlabel('M', fontsize=13)
        plt.ylabel('Error', fontsize=13)
        plt.legend(["Hardy","Franke","Modified Franke","NN"], fontsize=13,loc ="best")       

        plt.savefig(directory+'testcase'+str(no_testcase)+uniform+'.png',bbox_inches='tight', dpi=150) 
        plt.close()
    no_testcase+=1

[0.0005457961439072821, 0.015869834562494675, 0.0003619992932480622, 2.3181994002069853e-05]
[7.234479540100019e-05, 0.015594562326005586, 0.0002142706147488048, 2.5657601642160454e-05]
[6.547225279765243e-06, 0.017618864926487968, 0.00039475267544568946, 1.8099381702443366e-05]
[4.266750151042977e-06, 0.01758366135984368, 0.0001239066460835183, 9.762758546409688e-06]
[4.192721862838337e-06, 0.013026120086433772, 0.0016107777693503784, 6.741719598818553e-05]
[0.0007133047181155024, 0.023483628303861425, 0.0005317885306524334, 5.622123568499543e-05]
[4.6988248631547925e-05, 0.019243834285979998, 0.00024017472162462302, 1.2336757975124664e-05]
[1.4722847762786296e-05, 0.018072214048666862, 0.00027676358842852186, 2.3070388861230235e-05]
